# Unit 1: Working with Text data

## Unstructured data and vectorization

Folks apply machine learning to text for a number of different applications. Sentiment analysis, when you try to infer the mood of a sentence, is heavily used in marketing. Your Gmail's spam filter is working hard to protect you from all that evil stuff. Language identification, machine translation and topic detection are just a couple of other examples of how data science is applied to text data nowadays.

Many times machine learning algorithms are limited by the amount of data available for a given task. This is not the case for text, right? There is plenty of text data out there, especially since the Web shaped our lifes. Yet, machine learning struggled to provide near-humans results on text data for a long time. Why is that?

In machine learning, we describe examples, or data points, by a collection of certain features. A person can be described in terms of age, gender and profession, a song has name, album, gender, and so on, as many of the datasets you have seen so far. When data has this structure, we call it **structured data**.

In contract, text has no "natural" features. It's a sequence of signs that has somehow a meaning. Like images and sound, it is an example of **unstructured data**. While it's pretty easy for us to understand, machines find unstructured data surprisindly hard to process (*dumb machines, one point for us*). 

For this reason, we can not feed a machine learning classifier directly with text. We need to find a way to *make it structured*. When it comes to unstructured data, most approaches in machine learning solves the problem by finding a (good) way to squeeze the data into some form of vector space. Text is not exception.

This process, called **vectorization**, is key to obtaining good results when applying machine learning to text data. Much research has been done in this area and there are classical vectorization approaches. More recently, deep learning revolutioned this field (among others) providing very effective way to vectorize text. We treat some of the typical approaches in this unit, but `[SPOILER ALERT]` you will deal with some cool deep learning stuff in the last learning unit.

## Feature extraction

Ok, so, vectorization. Let's extract interesting features from text. How do we transform a document into a vector?

Firstly, a vector exists in a space. Our space will be an high-dimensional space corresponding to all the **keywords**. So, as a first attempt to build this space, we take each term appearing in the documents as a dimension, or a feature.

Some terminology: the collection of all the documents is the *corpus*. We call the *dictionary* the collection of distinct words occurring in the corpus.

![vector space](files/vector_space1.png)
*Note: due to our physical world unfortunate limitations, we can only draw examples in 3 dimensions. Imagine the same picture with thousands of dimensions.*

I know, time to see some code. 

Let's use the following text as a toy example. Those are examples of news headline from the [News Aggregator dataset](https://archive.ics.uci.edu/ml/datasets/News+Aggregator).

In [4]:
import pandas as pd
import numpy as np

health_document ='Men With Prostate Cancer May Not Always Get Better' # better start prevention early, guys
tech_document = "Facebook is always trying to alter people's behavior, says former data scientist. Facebook's Data Science team may have run hundreds of experiments without people's knowledge."
tv_document = "Game of Thrones Season 4 Spoilers, Rumors and News: Trailer Video is Sparking Questions, 'Who Will Get Vengeance And Who will Not?'"
docs = [health_document, tech_document, tv_document]

Now, we need to build our dictionary. As we saw a bit above, it is the collection of all the keywords in the corpus. 
Again, as a first attempt, let's pretend all words are keywords.

In [5]:
def build_dictionary():
    dictionary = set()

    for doc in docs:
        words = doc.split()
        dictionary.update(words)
    
    return dictionary

build_dictionary()

{"'Who",
 '4',
 'Always',
 'And',
 'Better',
 'Cancer',
 'Data',
 'Facebook',
 "Facebook's",
 'Game',
 'Get',
 'May',
 'Men',
 'News:',
 'Not',
 "Not?'",
 'Prostate',
 'Questions,',
 'Rumors',
 'Science',
 'Season',
 'Sparking',
 'Spoilers,',
 'Thrones',
 'Trailer',
 'Vengeance',
 'Video',
 'Who',
 'Will',
 'With',
 'alter',
 'always',
 'and',
 'behavior,',
 'data',
 'experiments',
 'former',
 'have',
 'hundreds',
 'is',
 'knowledge.',
 'may',
 'of',
 "people's",
 'run',
 'says',
 'scientist.',
 'team',
 'to',
 'trying',
 'will',
 'without'}

Note how Python's `set` object takes care of removing duplicates for words that appear more than once, like *"not"* and *"is"*. 

So, we have our dictionary. Those words are the dimensions of the feature space. 
We will transform the sentences to vectors (jargon: `project the sentences onto the vector space`) using a disturbingly simple approach: each feature, which correspond to a word, is equal to the count of the times the word appears in the document.

Easier to code than to say:

In [6]:
def vectorize():
    dictionary = build_dictionary()
    vectors = []
    for doc in docs:
        words = doc.split(' ')
        vector = np.array([doc.count(word) for word in dictionary])
        vectors.append(vector)
    
    return vectors

vectorize()

[array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]),
 array([0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 2, 0, 0, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 1,
        1, 0, 0, 0, 0, 0]),
 array([1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1])]

Congratulations, you have three vectors of integers representing your documents! 

Not very informative, though. It's just a bunch of numbers. Pandas can help us out here:

In [7]:
def build_df():
    return pd.DataFrame(vectorize(), columns=build_dictionary())

build_df()

,Game,Trailer,to,With,Not,people's,Facebook's,may,May,Season,...,of,Science,is,data,have,Thrones,News:,4,'Who,Will
0,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,2,1,1,0,0,...,1,1,2,1,1,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,1,...,1,0,1,0,0,1,1,1,1,1


Yes, much better. This DataFrame is way more similar to structured data than the initial documents. Still, there are a couple of things we can improve here.

## Feature selection

The dictionary contains both *Always* and *always*, some punctuation and even the number 4. Ugly, isn't it? A smart and easy improvement would be to convert all words to lowercase and strip numbers and punctuation signs.

In [8]:
import string
import re

docs = [re.sub(r'[\d+'+string.punctuation+']', '', doc.lower()) for doc in docs]

build_df()

,news,to,may,spoilers,science,hundreds,experiments,men,sparking,and,...,scientist,get,vengeance,is,with,data,peoples,behavior,have,trailer
0,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
1,0,1,1,0,1,1,1,1,0,0,...,1,0,0,2,1,2,2,1,1,0
2,1,0,0,1,0,0,0,0,1,2,...,0,1,1,1,0,0,0,0,0,1


Much cleaner, plus the space now is 44-dimensional, instead of 52-dimensional. Reducing dimensionality of data is always a good thing.

Taking a closer look to the DataFrame, it looks like most words just belong to one document. It would be a good signal, because they'd be very discriminative feature (*ok, at least in this toy example*). But there are other words that appear in more than one documents and aren't very informative: `to`, `not`, `is`, `with`.

Those words are called **stop words**. They are the most common words in a language, thus they usually appear in almost every documents of a corpus. Not adding much information, right? Let's strip them.

In [9]:
# The list came from here: http://snowball.tartarus.org/algorithms/english/stop.txt
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

def build_dictionary():
    dictionary = set()

    for doc in docs:
        words = [word for word in doc.split() if word not in stop_words]
        dictionary.update(words)
    
    return dictionary

def vectorize():
    dictionary = build_dictionary()
    vectors = []
    for doc in docs:
        words = doc.split(' ')
        vector = np.array([doc.count(word) for word in dictionary if word not in stop_words])
        vectors.append(vector)
    
    return vectors

build_df()

,news,may,spoilers,science,hundreds,experiments,men,sparking,trying,run,...,without,game,rumors,scientist,get,vengeance,data,peoples,behavior,trailer
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,1,0,1,1,1,1,0,1,1,...,1,0,0,1,0,0,2,2,1,0
2,1,0,1,0,0,0,0,1,0,0,...,0,1,1,0,1,1,0,0,0,1


## Feature weighting

When we compute the vectors just based on the counts of words inside documents, there is a subtle problem. Let's compute the length (AKA norm) of the vectors:

In [10]:
df = build_df()
df.sum(axis=1)

0     7
1    23
2    12
dtype: int64

The second vector has a **much** bigger norm than the other two. It's because the document contains more words. This may lead to misclassification due to the document length, and we don't want that.

Better normalize our vectors, dividing the counts for the document length. This representation is commonly referred as **Term Frequency (TF)**.

In [11]:
tf = df.div(df.sum(axis=1), axis=0)

Now vectors' norms are all equals and unitary:

In [12]:
tf.sum(axis=1)

0    1.0
1    1.0
2    1.0
dtype: float64

Much better for our classifier, but we are not done yet.

Not all the words has the same importance to find out what is the topic of a document. If a document is about technology, a word like _"data"_ is much more informative than _"always"_, and it should weight more toward a decision. In general, words that are very common in the corpus are less informative than rare words.

That is the rational behind **Inverse Document Frequency (IDF)**:
$$ IDF _{term} = log {\frac{\lvert D \lvert}{\lvert D_{term} \lvert}}   $$

where $D$ is the corpus, while $D_{term}$ is the subset of $D$ that contains $term$.

Combining TF and IDF, we define the values of our vectors as:

$$ TFIDF _{term} = TF _{term} * IDF _{term} $$

In short, we measure *the term frequency, weighted by its rarity in the entire corpus*, as perfectly put by Maria Dominguez during the 4th hackaton.

In [15]:
def idf(column):
    return np.log(len(column) / sum(column > 0))

tf_idf = tf.multiply(tf.apply(idf))
tf_idf

,news,may,spoilers,science,hundreds,experiments,men,sparking,trying,run,...,without,game,rumors,scientist,get,vengeance,data,peoples,behavior,trailer
0,0.000000,0.057924,0.000000,0.000000,0.000000,0.000000,0.057924,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.057924,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.017629,0.000000,0.047766,0.047766,0.047766,0.017629,0.000000,0.047766,0.047766,...,0.047766,0.000000,0.000000,0.047766,0.000000,0.000000,0.095532,0.095532,0.047766,0.000000
2,0.091551,0.000000,0.091551,0.000000,0.000000,0.000000,0.000000,0.091551,0.000000,0.000000,...,0.000000,0.091551,0.091551,0.000000,0.033789,0.091551,0.000000,0.000000,0.000000,0.091551


## Classifying text

Now that we have obtained a satisfying feature representation, it's probably time to feed a classifier with those features. Traditionally, good choices to classify text are Support Vector Machines and Bayesian classifiers. Due to its easy analytical tractability, let's choose a Naive Bayes classifier and see in details how it works.

Given $k$ classes $\{C_{1}, ..., C_{k}\}$ and a document $D$ to classify, our goal is to estimate the probabilistic model $P(C_{i} \vert D)$.

At classification time, we will choose the *maximum a posteriori*, or, in plain English, the most likely class. Computing that is simple: it will be the class $C^{*}$ such that:

$$C^{*} = argmax_{C_{i}} P(C_{i} \vert D)$$ 

There is still the issue to compute $P(C_{i} \vert D)$. Naive Bayes, as other Bayesian classifiers, uses Bayes' theorem to approach the problem: 

$$P(C_{i} \vert D) = \frac{P(D \lvert C_{i}) P(C_{i})}{P(D)}$$

Huum, does not look like a great step ahead. But let's have a deeper look to the formula.

The denominator $P(D)$ is the **evidence**. We can consider it constant, because it simply is our document, no uncertainty involved. Ok, let's forget about it.
$P(C_{i})$ is called the **prior**, and it does not depend on $D$. It can be treated as the probability of assigning class $C_{i}$ without even looking at the document. We can disregard it, right? 

We are left with the much more interesting $P(D \lvert C_{i})$, which is called the **likelihood**. In fact, it is the likelihood of the document $D$ to belong to class $C_{i}$. 

Remembering that we described a document in terms of its keywords $w_{j}$, this probability is equal to:

$$P(D \lvert C_{i}) = P(w_{1} \vert C_{i}, w_{2} \vert C_{i}, \dots,  w_{n} \vert C_{i})$$

Intuitively, you can imagine the classifier asking itself: *How is it likely that those words appear in a document about health?*, *How is it likely that those words appear in a document about technology?*, and so on.

Now, it is time for the **Naive assumptions**: each word $w_{j}$ is independent of the presence and the order of every other words. With these assumptions, the formula above is much more easily computable:

$$P(D \lvert C_{i}) = \prod_{j=1}^N P(w_{j} \lvert C_{i})$$

In theory, computing $P(w_{j} \lvert C_{i})$ is as easy as counting the occurrences of the word $w_{j}$ in documents of class $C_{i}$. In practice, it may produce extremely small numbers, though, leading to wrong results. An approximation that solves this problem is as follows:

$$P(w_{j} \lvert C_{i}) = \frac {occurrences \, of \, w_{j} \, in \, documents \, of \, C_{i} + 1} {total \, words \, in \, C_{i} + \lvert vocabulary \lvert} $$

That is easy to translate into Python:

In [20]:
def prob_word_given_class(word, C, train_set):
    word_in_class = sum(train_set.loc[train_set.target == C, word])
    total_words_in_class = sum((train_set[train_set.target == C] != 0).any(axis=1))
    dictionary_cardinality = train_set.shape[1]
    
    return (1 + word_in_class) / (total_words_in_class + dictionary_cardinality)

def prob_document_of_class(doc, C, train_set):
    preprocessed = re.sub(r'[\d+'+string.punctuation+']', '', doc.lower())
    words = pd.Series([word for word in preprocessed.split() if word in train_set.columns])
    probs = words.apply(lambda w: prob_word_given_class(w, C, train_set))
    return probs.prod()

That's all well and good, but we should apply all this math!

First things first. We turn our features into a real training set:

In [16]:
train_set = tf_idf.assign(target=['HEALTH', 'TECH', 'TV'])
train_set

,news,may,spoilers,science,hundreds,experiments,men,sparking,trying,run,...,game,rumors,scientist,get,vengeance,data,peoples,behavior,trailer,target
0,0.000000,0.057924,0.000000,0.000000,0.000000,0.000000,0.057924,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.057924,0.000000,0.000000,0.000000,0.000000,0.000000,HEALTH
1,0.000000,0.017629,0.000000,0.047766,0.047766,0.047766,0.017629,0.000000,0.047766,0.047766,...,0.000000,0.000000,0.047766,0.000000,0.000000,0.095532,0.095532,0.047766,0.000000,TECH
2,0.091551,0.000000,0.091551,0.000000,0.000000,0.000000,0.000000,0.091551,0.000000,0.000000,...,0.091551,0.091551,0.000000,0.033789,0.091551,0.000000,0.000000,0.000000,0.091551,TV


Then, let's try to classify a sample sentence:

In [21]:
new_document = 'Data science at Facebook seems fun!'

# Computing the probability for each class
probas = train_set.target.map(lambda C: prob_document_of_class(new_document, C, train_set))
probas

0    0.000020
1    0.000025
2    0.000020
Name: target, dtype: float64

In [22]:
predicted_class = train_set.target.iloc[probas.argmax()]
predicted_class

'TECH'

Awesome, right prediction! Well, it is a **very** toy example, but things seems to make sense.

Countexample: what would happen throwing some random text at the model? The desired results should be... no class predicted at all.

In [23]:
new_document = 'This is a very generic text, so I try to say nothing'
probas = train_set.target.map(lambda C: prob_document_of_class(new_document, C, train_set))
probas

0   NaN
1   NaN
2   NaN
Name: target, dtype: float64

![](https://media.apnarm.net.au/media/images/2016/03/21/babymeme_620x310-dsghuu4e57q2783lwl2_ct300x300.jpg)